# **First steps in QUEENS** 
***

---
First steps in QUEENS 

In [ ]:
import pqueens
import argparse
import os
import time
import sys
import numpy as np
try: import simplejson as json
except ImportError: import json

from  pqueens.resources.resource import parse_resources_from_configuration
from  pqueens.resources.resource import print_resources_status

from pqueens.database.mongodb import MongoDB

from collections import OrderedDict
from pqueens.emulators.gp_emulator import GPEmulator
from pqueens.resources.resource import parse_resources_from_configuration
from pqueens.resources.resource import print_resources_status
from pqueens.designers.designer_factory import DesignerFactory
from collections import OrderedDict



# set input params 
# path to input file in json format 
input_file = '/Users/jonas/work/adco/queens_code/pqueens/example_input_files/baci_config.json'
output_dir ='/Users/jonas/work/adco/queens_code/pqueens/example_input_files'
try:
    with open(input_file, 'r') as f:
        options = json.load(f, object_pairs_hook=OrderedDict)
except:
    raise Exception("config.json did not load properly.")
    
options["input_file"] = input_file
options["output_dir"] = output_dir
    
    

In [ ]:
def create_new_job(db,suggestion,experiment_name,options,resource_name):

    print("Created new job")
    jobs = load_jobs(db, experiment_name)

    job_id = len(jobs) + 1

    i = 0
    params = {}
    for key, variable_meta_data in options['variables'].items():
        params[key] = variable_meta_data
        params[key]['values']=suggestion[i]
        i+=1

    job = {
        'id'           : job_id,
        'params' :       params,
        'expt_dir'     : options['output_dir'],
        'expt_name'    : experiment_name,
        'resource'     : resource_name,
        'driver_type'  : options['driver']['driver_type'],
        'driver_params': options['driver']['driver_params'],
        'status'       : 'new',
        'submit time'  : time.time(),
        'start time'   : None,
        'end time'     : None
    }

    save_job(job, db, experiment_name)

    return job

def load_jobs(db, experiment_name):
    """load the jobs from the database

    Returns
    -------
    jobs : list
        a list of jobs or an empty list
    """
    jobs = db.load(experiment_name, 'jobs')

    if jobs is None:
        jobs = []
    if isinstance(jobs, dict):
        jobs = [jobs]

    return jobs

def save_job(job, db, experiment_name):
    """save a job to the database"""
    db.save(job, experiment_name, 'jobs', {'id' : job['id']})

def tired(db, experiment_name, resource):
    """
    return True if resources is not accepting jobs
    """
    jobs = load_jobs(db, experiment_name)
    if resource.accepting_jobs(jobs):
        return False
    return True

def build_emulator(db, experiment_name, options):
    jobs = load_jobs(db, experiment_name)
    num_jobs = len(jobs)
    num_param = len(options['variables'])
    # init arrays
    y = np.zeros((num_jobs,1))
    x = np.zeros((num_jobs,num_param))
    i = 0
    for job in jobs:
        y[i] = job['result']
        j=0
        print("job['params']{} ".format(job['params']))
        for param in job['params']:
            print("param{} ".format(param))
            x[i,j]=job['params'][param]['values']
            j+=1
            #params[key]['values']=suggestion[i]

        i+=1
    my_emulator = GPEmulator(x,y,options['variables'])
    mean_mean, var_mean = my_emulator.compute_mean()
    print('Mean{} '.format(mean_mean))
    print('Variance of Mean{} '.format(var_mean))

    return my_emulator


Make sure that MongoDB is actually running by typing
`ps -ef | grep mongod | grep -v grep | wc -l | tr -d ' '`

If not start it by

`mongod --dbpath <path to data directory>`

clear of necessary 
`mongo pqueens --eval "db.dropDatabase()"`

In [ ]:
# create resource
resources = parse_resources_from_configuration(options)

 # connect to the database
db_address = options['database']['address']
experiment_name = options['experiment-name']

sys.stderr.write('Using database at %s.\n' % db_address)
db   = MongoDB(database_address=db_address)

# create designer
designer = DesignerFactory.create_designer(options['designer']['type'],
                                           options['variables'],
                                           options['designer']['seed'],
                                           options['designer']['num_samples'])

suggester = designer.sample_generator()



Run Analysis

In [4]:
jobs = load_jobs(db, experiment_name)
for suggestion in suggester:
    processed_suggestion = False
    while not processed_suggestion:
        if resources['my-machine'].accepting_jobs(jobs):

            new_job = create_new_job(db,suggestion,experiment_name,options,'my-machine')

            # Submit the job to the appropriate resource
            process_id = resources['my-machine'].attempt_dispatch(experiment_name,
                                                                  new_job, db_address,
                                                                  options['output_dir'])

            # Set the status of the job appropriately (successfully submitted or not)
            if process_id is None:
                new_job['status'] = 'broken'
                save_job(new_job, db, experiment_name)
            else:
                print("suggested_job[status{} ".format(new_job['status']))
                new_job['status'] = 'pending'
                new_job['proc_id'] = process_id
                save_job(new_job, db, experiment_name)

            processed_suggestion = True
            jobs = load_jobs(db, experiment_name)
            print_resources_status(resources, jobs)

        else: # tired(db, experiment_name, resources['my-machine']):
            time.sleep(options.get('polling-time', 5))
            jobs = load_jobs(db, experiment_name)


        print_resources_status(resources, jobs)
#time.sleep(options.get('polling-time', 30))
#build_emulator(db, experiment_name, options)
# create emulator

Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000008.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 
Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000009.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 
Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000010.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 


Submitted job as process: 3157
Submitted job 8 with local scheduler (process id: 3157).

Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    3          5         
                *TOTAL*       3          5         


Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    3          5         
                *TOTAL*       3          5         

Submitted job as process: 3158
Submitted job 9 with local scheduler (process id: 3158).

Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    4          5         
                *TOTAL*       4          5         


Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    4          5         
                *TOTAL*       4          5         



Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000011.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 
Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000012.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 



Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    4          8         
                *TOTAL*       4          8         

Submitted job as process: 3192
Submitted job 13 with local scheduler (process id: 3192).

Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    5          8         
                *TOTAL*       5          8         


Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    5          8         
                *TOTAL*       5          8         



Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000013.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 


...


Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    5          15        
                *TOTAL*       5          15        


Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    0          20        
                *TOTAL*       0          20        

Submitted job as process: 3330
Submitted job 21 with local scheduler (process id: 3330).

Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    1          20        
                *TOTAL*       1          20        


Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    1          20        
                *TOTAL*       1          20        

Submitted job as process: 3331
Submitted job 22 with local scheduler (process id: 3331)

Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000021.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 
Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000022.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 
Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000023.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 
Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000024.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 
Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000025.out' mode='w' encoding='UTF-8'>



Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    5          20        
                *TOTAL*       5          20        


Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    4          21        
                *TOTAL*       4          21        

Submitted job as process: 3379
Submitted job 26 with local scheduler (process id: 3379).

Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    5          21        
                *TOTAL*       5          21        


Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    5          21        
                *TOTAL*       5          21        



Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000026.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 
Created new job
starting process
output_file<_io.TextIOWrapper name='/Users/jonas/work/adco/queens_code/pqueens/example_input_files/output/00000027.out' mode='w' encoding='UTF-8'>
suggested_job[statusnew 



Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    1          25        
                *TOTAL*       1          25        

Submitted job as process: 3405
Submitted job 27 with local scheduler (process id: 3405).

Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    2          25        
                *TOTAL*       2          25        


Resources:      NAME          PENDING    COMPLETE
                ----          -------    --------
                my-machine    2          25        
                *TOTAL*       2          25        



In [ ]:
    my_emulator = build_emulator(db, experiment_name, options)

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns # makes plots more pretty 
import numpy as np

my_pdf_hifi, y_plot_hifi  = my_emulator.compute_pdf()
fig = plt.figure()
fig.suptitle('Probability Density Function', fontsize=14)
line1, = plt.plot(y_plot_hifi, my_pdf_hifi['mean'], lw=2, color='blue',label='GP mean')
line2, = plt.plot(y_plot_hifi, my_pdf_hifi['median'], lw=2, color='green',label='GP median')
plt.fill_between(y_plot_hifi,my_pdf_hifi['quant_low'], my_pdf_hifi['quant_high'],
                  color='blue',alpha = 0.3,label='GP conf. reg.')

plt.ylabel('pdf(y)')
plt.xlabel('y')
plt.legend()
plt.show()


In [ ]:
cdf_val, sample_values = my_emulator.compute_cdf()

fig = plt.figure()
fig.suptitle('Cumulative Distribution Function', fontsize=14)
x = np.append(sample_values['q_lower_bound'],
              sample_values['q_upper_bound'][::-1])

y = np.append(cdf_val,cdf_val[::-1])


p = plt.Polygon(np.c_[x,y], color="blue",alpha=0.3,label='GP conf. reg.')
ax = plt.gca()
ax.add_patch(p)
line1, = plt.plot(sample_values['mean'], cdf_val, lw=2, color='blue',
                  label='GP mean')
line4, = plt.plot(sample_values['median'], cdf_val, lw=2, color='green',
                  label='GP median')

plt.legend()

plt.ylabel('cdf(y)')
plt.xlabel('y')
plt.show()

In [ ]:
y_quantile, my_fail_prob = my_emulator.compute_failure_probability_function()
fig = plt.figure()
fig.suptitle('Failure Probability', fontsize=14)
plt.ylabel('P_f(y_0)')
plt.xlabel('y_0')

line1, = plt.plot(y_quantile['mean'],my_fail_prob, lw=2, color='blue',label='GP mean')
line2, = plt.plot(y_quantile['median'],my_fail_prob, lw=2, color='green',label='GP median')

x = np.append(y_quantile['q_upper_bound'],
              y_quantile['q_lower_bound'][::-1])

y = np.append(my_fail_prob,my_fail_prob[::-1])

p = plt.Polygon(np.c_[x,y], color="blue",alpha=0.3,label='GP conf. reg.')
ax = plt.gca()
ax.add_patch(p)

plt.legend()
ax.set_yscale('log')
plt.show()